In [3]:
import spacy
import pandas as pd
import numpy as np

import Modules.solrhandler as sh
import Modules.clusterer as cls
import Modules.topicdeterminator as td
import Modules.chatbot as cb

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
#query = "anmeldung"
#target_service = 120686

In [22]:
# Import eval.csv
import os
file = os.path.realpath("../../Evaluierung/eval.csv")
eval = pd.read_csv(file, sep = ";")
eval = eval.head(20)
eval.head(20)

,documentId,file,dialogId,initialQuestion,suggestionCount,directlyFound,name
0,120697,20220131--quantEvalAllExcel.xlsx,4,Ummeldung,6,True,Änderung/Wechsel der Hauptwohnung
1,120686,20220131--quantEvalAllExcel.xlsx,4,Ummelden,6,True,Anmeldung einer Wohnung
2,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...
3,329421,20220131--quantEvalAllExcel.xlsx,19,Quarantäne Krankschreibung vom Arzt,6,True,Erstattung nach Infektionsschutzgesetz bei Tät...
4,120914,20220131--quantEvalAllExcel.xlsx,23,Termin für Zulassung Gebrauchtwagen,6,True,Zulassung eines Fahrzeuges mit auswärtigem Ken...
5,328200,20220131--quantEvalAllExcel.xlsx,24,Wo kann ich mich nach Corona Infektion freites...,6,True,Infektionskrankheiten melden - für Gemeinschaf...
6,330073,20220131--quantEvalAllExcel.xlsx,28,Ab wann ist man bei Johnson & Johnson geboostert,5,True,Impfung gegen Corona (SARS-CoV-2)
7,120702,20220131--quantEvalAllExcel.xlsx,36,Genesenen Bescheinigung,6,True,Meldebescheinigung beantragen
8,330150,20220131--quantEvalAllExcel.xlsx,39,"welche medikamente kann man nehmen, wenn man c...",6,True,Arzneimittel - zur Sachkundeprüfung für den Ha...
9,330174,20220131--quantEvalAllExcel.xlsx,47,ab 15.1. ist für Gäste von Restaurants Nachwe...,6,True,Gaststättengewerbe - zum Unterrichtungsnachwei...


In [6]:
eval.iloc[2]["initialQuestion"]

'Pcr test positiv brauche ich eine Krankschreibung vom Arzt für den Arbeitgeber'

Funktionen

In [90]:
def findCorrectAnswer(chatbot,targetService):

    # Find Target Cluster
    clusteredColumn = chatbot.clusterer.getClusteredColumn()
    service = chatbot.df.loc[chatbot.df["id"]==str(targetService)]
    if len(service[clusteredColumn].values) == 0:
        return None

    targetCluster = service[clusteredColumn].values[0]

    if targetCluster == chatbot.getSelectedClusterForQuestion():
        return True

    else:
        return False

def getRank(chatbot, targetService):
    temp = chatbot.df.reset_index()
    try:
        rank = temp.index[temp["id"]==str(targetService)].values[0]
    except IndexError as e:
        return float("NaN")
    return rank + 1

Logging

In [91]:
class Logger:
    
    def __init__(self, eval_df):
        self.liste = []
        self.eval_df = eval_df
        self.chatbot = None
    
    def getDataFrame(self): # TODO
        pass
    
    def setChatbot(self, chatbot):
        self.chatbot = chatbot
    
    def log(self, ID, t, row, answer = None, initialQuery = False): 
        target_service = row["documentId"]
        
        dialogId = row["dialogId"]
        file = row["file"]
        
        service_name = row["name"]
        query = row["initialQuestion"]
        
        if initialQuery:
            nResults = len(self.chatbot.df.index)
            rank = getRank(self.chatbot,target_service)# getRank(target_service)

            row = (file, dialogId, ID, t, service_name, query, "initialQuery", None, rank, nResults)
        else:
            if answer is not None:


                nResults = len(self.chatbot.df.index)
                question = self.chatbot.generateQuestion()
                answer = answer

                rank = getRank(self.chatbot,target_service)# getRank(target_service)

                row = (file, dialogId, ID, t,  service_name, query, question, answer, rank, nResults)
            else:
                row = (file, dialogId, ID, t, service_name, query, None, None, None, None)
        self.liste.append(row)
        #print(self.liste)

Execution

In [92]:
# Initialisierung Komponenten Chatbot
solrhandler = sh.SolrHandler(20)
clusterer = cls.Clusterer()
tpc_dterminator = td.TopicDeterminator()

In [108]:
# Experiment durchlaufen und Loggen
lg = Logger(eval_df = eval)   

for (ID, row) in eval.iterrows():
    
    query = row["initialQuestion"]
    target_service = row["documentId"]
    
    # Chatbot initialisieren
    try:
        chatbot = cb.Chatbot(query,3,solrhandler,clusterer,tpc_dterminator)
    
    # Fehler überspringen: eg. kein solr output
    except Exception as e: 
        if(str(e) == 'response'): # TODO ??
            next
        elif(str(e) == 'no solr output'):
            next
        else:
            print(e)
            
    lg.setChatbot(chatbot)
    t = 0
    answer = findCorrectAnswer(chatbot, target_service)
    lg.log(ID,t, row, answer, initialQuery = True)
    if answer is None:
        continue
    
    
    while not chatbot.isFinished():#chatbot.refineResultset(answer, recluster = True):

        # log
        t += 1
        
        answer = findCorrectAnswer(chatbot, target_service)
        chatbot.refineResultset(answer, True)
        if answer is None: 
            break
        lg.log(ID, t, row,answer)

'response'


In [113]:
log_df = pd.DataFrame(lg.liste, columns=["file","dialogId", "ID","t", "name", "query", "question", "answer", "rank", "nResult"])
#log_df.head(20)

Log Analyzer

In [114]:
import Modules.log_analyzer as lo_an

In [116]:
log_analyzer=lo_an.Log_Analyzer(log_df)
delta,services,questions,means=log_analyzer.run_it()

In [117]:
log_df

,file,dialogId,ID,t,name,query,question,answer,rank,nResult,nResult_reduction,delta_nResults,delta_Rank
0,20220131--quantEvalAllExcel.xlsx,4,0,0,Änderung/Wechsel der Hauptwohnung,Ummeldung,initialQuery,None,3.0,7,NaN,NaN,NaN
1,20220131--quantEvalAllExcel.xlsx,4,0,1,Änderung/Wechsel der Hauptwohnung,Ummeldung,Geht es bei ihrem Anliegen um zulassung?,True,2.0,5,0.285714,2.0,1.0
2,20220131--quantEvalAllExcel.xlsx,4,1,0,Anmeldung einer Wohnung,Ummelden,initialQuery,None,NaN,7,NaN,NaN,NaN
3,20220131--quantEvalAllExcel.xlsx,19,2,0,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,initialQuery,None,4.0,13,NaN,NaN,NaN
4,20220131--quantEvalAllExcel.xlsx,19,2,1,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um sars?,False,4.0,6,0.538462,7.0,0.0
5,20220131--quantEvalAllExcel.xlsx,19,2,2,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um hinweiscrn?,True,2.0,3,0.500000,3.0,2.0
6,20220131--quantEvalAllExcel.xlsx,19,3,0,Erstattung nach Infektionsschutzgesetz bei Tät...,Quarantäne Krankschreibung vom Arzt,initialQuery,None,1.0,10,NaN,NaN,NaN
7,20220131--quantEvalAllExcel.xlsx,19,3,1,Erstattung nach Infektionsschutzgesetz bei Tät...,Quarantäne Krankschreibung vom Arzt,Geht es bei ihrem Anliegen um cov?,False,1.0,3,0.700000,7.0,0.0
8,20220131--quantEvalAllExcel.xlsx,23,4,0,Zulassung eines Fahrzeuges mit auswärtigem Ken...,Termin für Zulassung Gebrauchtwagen,initialQuery,None,7.0,20,NaN,NaN,NaN
9,20220131--quantEvalAllExcel.xlsx,23,4,1,Zulassung eines Fahrzeuges mit auswärtigem Ken...,Termin für Zulassung Gebrauchtwagen,Geht es bei ihrem Anliegen um kfz?,True,7.0,14,0.300000,6.0,0.0


Statistics ( Alt)

In [96]:
# avg Rank Improvement in turn t
log_df["delta_Rank"] = log_df.groupby(["ID"])["rank"].diff()
#log_df.groupby(["t"])["rank_improvement"].mean()

In [97]:
# avg nSuggestions Improvement in turn t
log_df["delta_nResults"] = log_df.groupby(["ID"])["nResults"].diff()
#log_df.groupby(["t"])["sugg_improvement"].mean()

In [ ]:
# MRR ohne cluster refinement
#df = log_df.loc[log_df["t"] == 1]
#MRR_ohne = 1/(df["rank"].mean())
#MRR_ohne

In [ ]:
# MRR mit cluster refinement
#df = log_df.groupby(["ID"]).last("t")
#MRR_mit = 1/(df["rank"].mean())
#MRR_mit
#df.head()

In [99]:
# avg Turns
print(log_df["t"].mean())

# bei durchschnittlich nResults
print(log_df.groupby(["ID"]).last("t")["nResults"].mean())

0.4375
4.3


In [98]:
log_df.head(20)

,file,dialogId,ID,t,service_name,query,question,answer,rank,nResults,delta_Rank,delta_nResults
0,20220131--quantEvalAllExcel.xlsx,4,0,0,Änderung/Wechsel der Hauptwohnung,Ummeldung,initialQuery,None,3.0,7,NaN,NaN
1,20220131--quantEvalAllExcel.xlsx,4,0,1,Änderung/Wechsel der Hauptwohnung,Ummeldung,Geht es bei ihrem Anliegen um zulassung?,True,2.0,5,-1.0,-2.0
2,20220131--quantEvalAllExcel.xlsx,4,1,0,Anmeldung einer Wohnung,Ummelden,initialQuery,None,NaN,7,NaN,NaN
3,20220131--quantEvalAllExcel.xlsx,19,2,0,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,initialQuery,None,4.0,13,NaN,NaN
4,20220131--quantEvalAllExcel.xlsx,19,2,1,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um sars?,False,4.0,6,0.0,-7.0
5,20220131--quantEvalAllExcel.xlsx,19,2,2,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um hinweiscrn?,True,2.0,3,-2.0,-3.0
6,20220131--quantEvalAllExcel.xlsx,19,3,0,Erstattung nach Infektionsschutzgesetz bei Tät...,Quarantäne Krankschreibung vom Arzt,initialQuery,None,1.0,10,NaN,NaN
7,20220131--quantEvalAllExcel.xlsx,19,3,1,Erstattung nach Infektionsschutzgesetz bei Tät...,Quarantäne Krankschreibung vom Arzt,Geht es bei ihrem Anliegen um cov?,False,1.0,3,0.0,-7.0
8,20220131--quantEvalAllExcel.xlsx,23,4,0,Zulassung eines Fahrzeuges mit auswärtigem Ken...,Termin für Zulassung Gebrauchtwagen,initialQuery,None,7.0,20,NaN,NaN
9,20220131--quantEvalAllExcel.xlsx,23,4,1,Zulassung eines Fahrzeuges mit auswärtigem Ken...,Termin für Zulassung Gebrauchtwagen,Geht es bei ihrem Anliegen um kfz?,True,7.0,14,0.0,-6.0
